In [41]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 24.1 kB/s eta 0:00:0000:0100:14
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.6 kB/s eta 0:00:0000:0100:14
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
?Q:为什么说平均化注意力加权的位置降低了有效的resolution A:各个head可能会关注输入的不同方面，随机分布能够保证后续的学习到的特征是互相远离，而不是重合https://blog.csdn.net/chattyfish/article/details/130234363
?Q:decode为什么需要src_mask A:在decoderlayer的forward中做入参并作软寻址时使用
?Q:sublayer怎么用 A:SublayerConnection预置layernorm和残差连接并将layer作为入参复用 妙啊
?Q:encoder中mask和decoder中mask合并写法怎么理解 A:
?Transformer参数量计算:

In [9]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt # 可视化库
from torchtext.data.functional import to_map_style_dataset
import spacy
import GPUtil
    rt warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


warnings.filterwarnings("ignore")
RUN_EXAMPLES = True # 设为False则跳过notebook执行

In [10]:
# 测试log_softmax，即在softmax处理后加入log对数函数处理，注意torch.nn和torch.nn.functional的区别
from icecream import ic
from torch.nn.functional import log_softmax, softmax

a = torch.rand(2,3)
ic(a)
b = softmax(a, dim=1)  

ic(b)
c = log_softmax(a, dim=1)
ic(c)

ic| a: tensor([[0.8456, 0.5370, 0.7207],
               [0.6593, 0.4252, 0.6260]])
ic| b: tensor([[0.3821, 0.2806, 0.3372],
               [0.3625, 0.2868, 0.3506]])
ic| c: tensor([[-0.9620, -1.2707, -1.0869],
               [-1.0147, -1.2488, -1.0480]])


tensor([[-0.9620, -1.2707, -1.0869],
        [-1.0147, -1.2488, -1.0480]])

In [11]:
# 快捷辅助函数
def is_interactive_notebook():
    return __name__ == "__main__"

def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)

def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)

class DummyQptimizer(torch.optim.Optimizer):
    def __init_(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self):
        None

    def zero_grad(self, set_to_none=False):
        None

class DummyScheduler:
    def step(self):
        None

In [12]:
class EncoderDecoder(nn.Module):
    """
    标准Encoder-Decoder结构
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "接收和处理掩码输入和目标序列"
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

class Generator(nn.Module):
    """
    定义标准线性+softmax生成步骤
    """
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [13]:
def clones(module, N):
    "生成多个相同的层"
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class LayerNorm(nn.Module):
    """
    LayerNorm模块，每条样本的所有特征归一化，区别于BatchNorm中的每个batch进行归一化
    """
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [14]:
class Encoder(nn.Module):
    """
    核心encoder是一个N层的堆叠
    """
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "让输入和mask依次通过每一层"
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [15]:
class SublayerConnection(nn.Module):
    """
    子层连接模块，在两个子层中每个子层应用残差连接并进行层归一化
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "将残差连接应用于每一个相同大小的子层"
        return x + self.dropout(sublayer(self.norm(x)))

In [16]:
class EncoderLayer(nn.Module):
    """
    编码器层，由两个子层组成，自注意层和前馈层
    """
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "encoder前向网络构建"
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [17]:
class Decoder(nn.Module):
    """
    带有mask的N层解码器层
    """
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [18]:
class DecoderLayer(nn.Module):
    """
    解码器层，由自注意力层、源注意力层和前馈层构成
    """
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "decoder前向网络构建"
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [19]:
def subsequent_mask(size):
    """
    屏蔽掉后续的位置
    注意掩码下面显示了每个tgt单词(行)被允许查看的位置(列)。在训练过程中，单词会被屏蔽，以便关注将来的单词。
    """
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

In [20]:
# 列表推导式 + alt绘图
def example_mask():
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Subsequent Mask": subsequent_mask(20)[0][x, y].flatten(),
                    "Window": y,
                    "Masking": x,
                }
            )
            for y in range(20)
            for x in range(20)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect()
        .properties(height=250, width=250)
        .encode(
            alt.X("Window:O"),
            alt.Y("Masking:O"),
            alt.Color("Subsequent Mask:Q", scale=alt.Scale(scheme="viridis")),
        )
        .interactive()
    )


show_example(example_mask)

alt.Chart(...)

In [21]:
def attention(query, key, value, mask=None, dropout=None):
    "计算归一化点乘注意力权值：Scaled Dot Product Attention（SDPA）"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [22]:
class MultiHeadedAttention(nn.Module):
    """
    多头注意允许模型在不同位置共同注意来自不同表示子空间的信息。对于单一注意力头，平均会抑制这一点
        1）解码器中的每个位置都能获取输入序列中的所有位置
        2）编码器中的每个位置都可以处理编码器前一层中的所有位置
        3）解码器中的每个位置能获取解码器之前的所有位置直至并包括该位置（-∞）
    """
    def __init__(self, h, d_model, dropout=0.1):
        "初始化模型size和head头个数"
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # 假定d_v = d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "实现多注意力头"
        if mask is not None:
            # 对所有h个head应用相同mask策略
            mask = mask.unsqueeze(1) # 增加一维
        nbatches = query.size(0)

        # 1) 多头qkv映射，其中linears只用前三层
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]
        # 2) 应用attention到batch中的所有映射向量 
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        # 3) 用view进行拼接并且应用最后一个线性映射，注意contiguous写法
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        del query, key, value
        return self.linears[-1](x)

In [23]:
# query, key, value = [torch.randn(8, 512, 512)] * 3
# h = 8
# nbatches = query.size(0)
# d_model = query.size(1)
# d_k = d_model // h
# mask = None
# dropout = 0.1

# linears = clones(nn.Linear(d_model, d_model), 4)
# query, key, value = [
#             lin(x).view(nbatches, -1, h, d_k).transpose(1, 2)
#             for lin, x in zip(linears, (query, key, value))
#         ]

# x, attn = attention(query, key, value, mask=mask, dropout=nn.Dropout(p=dropout))
# x = x.transpose(1, 2).contiguous().view(nbatches, -1, h * d_k)
# linears[-1](x).shape

In [25]:
class PositionwiseFeedForward(nn.Module):
    "实现FFN前馈神经网络"
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [26]:
class Embeddings(nn.Module):
    """嵌入层共享参数"""
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        # 注意在embedding层会乘以√d_model
        return self.lut(x) * math.sqrt(self.d_model)

In [40]:
class PositionalEncoding(nn.Module):
    """
    实现PE功能，尝试过参数学习版本的PE，但是最终选择了正弦版本
    因为它可以允许模型推断出比训练期间遇到的序列长度更长的序列长度
    """
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 在对数空间中计算一次位置encodings
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        # pe-tensor中0维代表batchsize，1维代表token，2维代表embedding
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [27]:
def example_positional():
    pe = PositionalEncoding(20, 0)
    y = pe.forward(torch.zeros(1, 100, 20))

    data = pd.concat(
        [
            pd.DataFrame(
                {
                    "embedding": y[0, :, dim],
                    "dimension": dim,
                    "position": list(range(100)),
                }
            )
            for dim in [4, 5, 6, 7]
        ]
    )

    return (
        alt.Chart(data)
        .mark_line()
        .properties(width=800)
        .encode(x="position", y="embedding", color="dimension:N")
        .interactive()
    )


show_example(example_positional)

alt.Chart(...)

In [50]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    """构建指定超参数模型"""
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # 重要设置：通过Glorot / fan_avg 初始化参数
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [77]:
'''获取模型结构并输出模型参数'''
def model_structure(model):
    blank = ' '
    print('-' * 90)
    print('|' + ' ' * 11 + 'weight name' + ' ' * 10 + '|' \
          + ' ' * 15 + 'weight shape' + ' ' * 15 + '|' \
          + ' ' * 3 + 'number' + ' ' * 3 + '|')
    print('-' * 90)
    num_para = 0
    type_size = 1  # 如果是浮点数就是4

    for index, (key, w_variable) in enumerate(model.named_parameters()):
        if len(key) <= 30:
            key = key + (30 - len(key)) * blank
        shape = str(w_variable.shape)
        if len(shape) <= 40:
            shape = shape + (40 - len(shape)) * blank
        each_para = 1
        for k in w_variable.shape:
            each_para *= k
        num_para += each_para
        str_num = str(each_para)
        if len(str_num) <= 10:
            str_num = str_num + (10 - len(str_num)) * blank

        print('| {} | {} | {} |'.format(key, shape, str_num))
    print('-' * 90)
    print('The total number of parameters: ' + str(num_para))
    print('The parameters of Model {}: {:4f}M'.format(model._get_name(), num_para * type_size / 1000 / 1000))
    print('-' * 90)


In [110]:
test_model = make_model(11, 11, 2)
test_model.eval()
src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
src_mask = torch.ones(1, 1, 10)
memory = test_model.encode(src, src_mask)
memory.shape
ys = torch.zeros(1, 1).type_as(src)
out = test_model.decode(memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data))
print(out.shape)
prob = test_model.generator(out[:, -1])
print(prob.shape)
_, next_word = torch.max(prob, dim=1)
next_word = next_word.data[0]
next_word

torch.Size([1, 1, 512])
torch.Size([1, 11])


tensor(4)

In [108]:
'''构建推理demo，输入为1-10数字，生成9个word并与原始y拼接'''
def inference_test():
    test_model = make_model(11, 11, 2)
    # model_structure(test_model)
    test_model.eval()
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    src_mask = torch.ones(1, 1, 10)

    memory = test_model.encode(src, src_mask)
    ys = torch.zeros(1, 1).type_as(src)

    for i in range(9):
        out = test_model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = test_model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )

    print("Example Untrained Model Prediction:", ys)


def run_tests():
    for _ in range(10):
        inference_test()


show_example(run_tests)

Example Untrained Model Prediction: tensor([[0, 8, 4, 0, 8, 8, 8, 9, 8, 9]])
Example Untrained Model Prediction: tensor([[ 0,  4,  0,  4,  0,  4, 10,  4,  0,  8]])
Example Untrained Model Prediction: tensor([[0, 4, 2, 5, 2, 5, 2, 5, 2, 5]])
Example Untrained Model Prediction: tensor([[0, 5, 2, 3, 0, 5, 3, 3, 3, 2]])
Example Untrained Model Prediction: tensor([[0, 2, 2, 2, 2, 2, 2, 2, 2, 2]])
Example Untrained Model Prediction: tensor([[0, 8, 2, 2, 2, 2, 2, 2, 2, 2]])
Example Untrained Model Prediction: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
Example Untrained Model Prediction: tensor([[0, 3, 8, 0, 3, 8, 0, 3, 8, 1]])
Example Untrained Model Prediction: tensor([[0, 1, 8, 1, 8, 1, 8, 5, 1, 8]])
Example Untrained Model Prediction: tensor([[ 0,  3,  7,  3,  0,  5,  1, 10,  1,  7]])
